# Volumetric Stiffness Field Playground

In [1]:
# Auto reload modules
%load_ext autoreload
%autoreload 2

#if you haven't installed vsf and just want to run this from the current directory, uncomment the following lines
import sys
sys.path.append('..')

import numpy as np
from vsf import vsf_from_file
from klampt.io import open3d_convert
from klampt import vis, Geometry3D, WorldModel
from vsf.visualize.klampt_visualization import vsf_show

vis.init('PyQt')  #needed to pop up Klampt OpenGL windows in Jupyter notebook
None

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
***  klampt.vis: using Qt6 as the visualization backend  ***


### Download the demo data from box

    Demo data: https://uofi.box.com/s/31wozq63qgqvg8012r1jdfj5mdchmmfp

<!-- + #### Download the full dataset
    + Download objects assets: https://uofi.box.com/s/2wj11zze45nj2owwqdpm7tsh0keleedw
    + Download estimates VSF models for simulation: https://uofi.box.com/s/g1i02pxf4quk0zt7nzv4qhovlhfnt3bl
    + Download raw visual-tactile dataset: https://uofi.box.com/s/grqsjh5m27jkin14t097pvjvy9b1bk6r -->

## Loading a VSF from file

In this example we will load one of the demo objects.  Let's assume that you have placed `demo_data` in the parent directory of this Jupyter notebook.

In [ ]:
vsf_model = vsf_from_file('../demo_data/saved_vsfs/rubber_fig_tall_angle00/vsf.npz')
print('Model loaded')

Now we can visualize the loaded model.  We can do so in Open3D using a point cloud visualization, or in Klampt with a point cloud or implicit surface visualization.

In [ ]:
# Visualize VSF point cloud in Open3D
import open3d as o3d
from vsf.visualize.o3d_visualization import vsf_to_point_cloud
vis_pcd = vsf_to_point_cloud(vsf_model)
o3d.visualization.draw_geometries([vis_pcd], window_name='VSF vis_pcd')

In [ ]:
# Klampt point cloud visualization of VSF
from vsf.visualize.klampt_visualization import vsf_to_point_cloud
k_pcd = Geometry3D(vsf_to_point_cloud(vsf_model))
vis.debug(k_pcd)

In [ ]:
# Klampt volumetric visualization of VSF
stiffness_levels = [0.27072093, 4.54502328, 8.81932563, 9.09362797, 10.36793032]
vsf_show(vsf_model, stiffness_levels)

## VSF estimation from tactile data

In this example, we will create VSF from a dataset that includes tactile observations.

- Create an empty VSF.  We will use a VSFFactory to shape the VSF rest points from an RGBD point cloud
- Configure the robot model and sensor(s).  In this case the robot is a Kinova Gen3, and there is only one sensor that provides joint torques.
- Load a dataset of trials giving robot actions and observations from sensor(s) 
- Create an estimator, and run it in batch mode on the dataset.

Let's start by creating an empty VSF.  If you have an RGB-D scanned object, you should use a `VSFRGBDCameraFactory` like the following code to construct the VSF points from a region of interest in the point cloud, extending through a volume behind the camera's point of view.  It doesn't really matter how you get the points; you can also create an empty box using the `vsf.vsf_from_box()` function, create a VSF from a 3D scanned mesh using `vsf.vsf_from_mesh()`, or load a previously saved VSF from disk.

In [ ]:
from vsf.core.vsf_factory import VSFRGBDCameraFactory, VSFRGBDCameraFactoryConfig, ViewConfig

view = ViewConfig(origin=[-0.55639158,1.04689234,0.52593784])  # need to set the origin of the camera so that volume points can be created
#we will select points within a bounding box 
config = VSFRGBDCameraFactoryConfig(features=['color'], voxel_size=0.02,
                                    view = view, bbox = [[-1.03, -0.64, -0.22], [-0.29, 0.23, 0.73]], downsample_visual=True, verbose=True)
factory = VSFRGBDCameraFactory(config)
#process() does all the work
vsf_empty = factory.process('../demo_data/datasets/rubber_fig_tall_angle00/object/bg_pcd.pcd')  #the PCD file is the point cloud of the RGBD scene, including the background 
print('{} rest points for VSF model created from RGBD factory'.format(len(vsf_empty.rest_points)))

In [ ]:
# Show the processed VSF model
from vsf.visualize.klampt_visualization import vsf_to_point_cloud
vis.debug(empty_pc = vsf_to_point_cloud(vsf_empty, masked_view_fraction=1.0))

Next, we create a world with a robot, a simulator, and one or more sensors.  This example loads a Kinova Gen3 robot model and uses the joint torque sensors in the simulator.  We also add the empty VSF into the simulator using `sim.add_deformable`.  This creates a body that can be moved through space.  (By default, all VSFs are considered to have a static pose, which works well if the object doesn't move much during interaction.  We will consider moving objects later.)

In [ ]:
from vsf.sim import klamptWorldWrapper, QuasistaticVSFSimulator
from vsf.sim.point_vsf_body import ContactParams
from vsf.sensor.joint_torque_sensor import JointTorqueSensor

#create a world
world = klamptWorldWrapper()
world.add_robot('kinova','../knowledge/robot_model/kinova_gen3.urdf')
robot = world.world.robot(0)

#this preprocessing needs to be done before running the simulator with a point VSF body
world.setup_local_pcd_lst('open3d')
world.setup_local_sdf_lst()

#create a simulator with the world, a joint torque sensor, and the vsf body
sensor = JointTorqueSensor('kinova_joint_torques','kinova',[robot.link(i).name for i in range(1,8)])
sim = QuasistaticVSFSimulator(world, [sensor])
vsf_body = sim.add_deformable('vsf',vsf_empty, contact_params=None)  #if you want to customize how the VSF is simulated, you can pass in a ContactParams object here

In [ ]:
# show the world and VSF
from vsf.visualize.klampt_visualization import vsf_to_point_cloud
vis.debug(world=world.world, empty_pc = vsf_to_point_cloud(vsf_empty), origin = view.origin)

Now we will set up a dataset. We will load a dataset from a standard folder format, which is configured and loaded automatically in the following example.  We will use a `DatasetConfig` which specifies which keys are present, and which keys used for robot commands and sensor observations.  The `dataset_from_config` function will set up a dataset appropriately to follow the specified configuration.  The standard dataset lazy-loads sequences into memory.

More generally, a dataset is any object that can be treated like a list of sequences, and each sequence is a list of dictionaries mapping keys to numpy arrays.  In other words, datasets behave like a `List[List[Dict[str,np.ndarray]]]` type. If you don't want to go through the trouble of saving such a thing to disk, you can just create such an object yourself.  

In [ ]:
from vsf.dataset.constructors import DatasetConfig, dataset_from_config

keys = {'kinova_joint_torques':7,'kinova_state':7}  #describes the keys present in the dataset
dataset_config = DatasetConfig(type='joint_torque_dataset', 
                               path='../demo_data/datasets/rubber_fig_tall_angle00/split1',
                               keys=keys,
                               sensor_keys={'kinova_joint_torques':'kinova_joint_torques'},
                               control_keys={'kinova':'kinova_state'})
dataset = dataset_from_config(dataset_config)
print("Dataset has {} sequences".format(len(dataset)))

For debugging, let's just validate the trajectories coming from the dataset.  Here we'll show the first 5 trajectories.

In [ ]:
from klampt.model.trajectory import RobotTrajectory 

for seqno in range(len(dataset)):
    #extract the sequence of commands
    seq = dataset[seqno]
    commands = []
    for frame in range(len(seq)):
        frame = seq[frame]
        commands.append(frame['kinova_state'])

    #convert to a RobotTrajectory and show it
    configs = [robot.configFromDrivers(d) for d in commands]
    traj = RobotTrajectory(robot,[i/len(configs) for i in range(len(configs))],configs)
    vis.debug(robot, {'animation':traj}, 'empty_pc', vsf_to_point_cloud(vsf_empty))
    if seqno >= 4: break

We can also check the discrepancy between the observations and the predicted measurements for a random VSF stiffness.  Here, we're also introducing the notion of sensor calibration, which is usually necessary at the beginning of a trial to tare the values coming from the sensor.

A `BaseSensor` can potentially handle its own calibration, but we find it's easier to separate how to calibrate the sensor from the sensor simulator itself.  Here we will generate a `BaseCalibrator` object that we will run at the beginning of each trial to calibrate the sensor.

(There is a little bit of work here to convert the dataset keys to the control and sensor keys expected by the simulator, calibrator, and estimators. This boilerplate is used a lot...)

In [ ]:
from vsf.sensor import TareCalibrator
import numpy as np

vsf_empty.stiffness.fill_(0.1)  #set a guessed stiffness of the VSF model
calibrator = TareCalibrator()
for seqno in range(len(dataset)):
    #extract the sequence of controls and observations.  This is boilerplate
    seq = dataset[seqno]
    control_seq = []
    sensor_seq = []
    for frame in seq:
        control_seq.append({k:frame[v] for k,v in dataset_config.control_keys.items()})
        sensor_seq.append({k:frame[v] for k,v in dataset_config.sensor_keys.items()})
        
    #run the calibration
    sim.reset()
    n = calibrator.calibrate(sensor,sim,control_seq,sensor_seq)
    #returns the # of samples used in calibration.  Technically should skip this number of frames for estimation
    print("Sequence",seqno,"calibration:",sensor.get_calibration())

    #now, run the simulator and compare the predicted torques to the actual torques
    dt = 0.1  # a guessed time step.  There's no time-dependent functionality in the quasistatic simulator, so this doesn't matter
    diffs = []
    for frameno in range(n,len(seq)):
        sim.step(control_seq[frameno],dt)
        pred = sim.measurements()['kinova_joint_torques']
        actual = sensor_seq[frameno]['kinova_joint_torques']
        assert len(pred) == len(actual)
        diffs.append(pred-actual)
    diffs = np.array(diffs)
    print("Sequence",seqno,"joint torque RMSEs",np.sqrt(np.mean(diffs**2,axis=0)))

    if seqno >= 4: break


Now, let's start the estimation process.  Let's start by using *batch* estimation, which uses all the items in the dataset to estimate the VSF stiffness parameters.  We have to provide a prior to the estimator to initialize its stiffness guess, and we will use a GaussianVSFPriorFactory which assigns an independent Gaussian distribution to each point's stiffness.

We can also customize the configuration of the estimator, such as the optimization technique used, but let's not worry about this too much for now.

In [ ]:
from vsf.sensor import TareCalibrator
from vsf.estimator.point_vsf_estimator import PointVSFEstimator, PointVSFEstimatorConfig
from vsf.prior.prior_factory import GaussianVSFPriorFactory
import time
import copy
vsf_est = vsf_empty
vsf_empty = copy.deepcopy(vsf_est)  #save a copy of the uninitialized VSF for later

#the second argument sets the prior estimate to a mean of 0.1 and a standard deviation of 1.0
estimator = PointVSFEstimator(PointVSFEstimatorConfig(), GaussianVSFPriorFactory(0.1,1.0))

t0 = time.time()
estimator.batch_estimate(sim, vsf_est, [dataset[i] for i in range(5)], dataset_config, {sensor.name:TareCalibrator()})
t1 = time.time()
print("Estimation took time",t1-t0)

Now let's see some statistics about the estimates.  The VSF stiffness values are in the `stiffness` attribute, and it will also have new two features, 'K_std' and 'N_obs', which give the standard deviations of the estimates and the number of frames in which each point has been touched, respectively.

In [ ]:
print('Features:',list(vsf_est.features.keys()))
stiffness = vsf_est.stiffness.cpu().numpy()
N_obs = vsf_est.features['N_obs'].cpu().numpy()
touched_mask = N_obs > 0
print('Stiffness min {}, max {}, mean {}, std {}'.format(np.min(stiffness),np.max(stiffness),np.average(stiffness),np.std(stiffness)))
print('Number of points touched {}/{}'.format(np.sum(N_obs > 0),stiffness.shape[0]))
print('Touched stiffness min {}, max {}, mean {}, std {}'.format(np.min(stiffness[touched_mask]),np.max(stiffness[touched_mask]),np.average(stiffness[touched_mask]),np.std(stiffness[touched_mask])))


Now let's visualize the estimates. For clearer visualization, we will set the stiffness of the untouched points to 0.

In [ ]:
vsf_est.stiffness[~touched_mask] = 0
vis.debug(world=world.world, estimated_pc = vsf_to_point_cloud(vsf_est), origin = view.origin)

Finally, let's check the accuracy of the predictions on the 5-sequence training set.

In [ ]:
for seqno in range(len(dataset)):
    #extract the sequence of controls and observations.  This is boilerplate
    seq = dataset[seqno]
    control_seq = []
    sensor_seq = []
    for frame in seq:
        control_seq.append({k:frame[v] for k,v in dataset_config.control_keys.items()})
        sensor_seq.append({k:frame[v] for k,v in dataset_config.sensor_keys.items()})
        
    #run the calibration
    sim.reset()
    n = calibrator.calibrate(sensor,sim,control_seq,sensor_seq)

    #now, run the simulator and compare the predicted torques to the actual torques
    dt = 0.1  # a guessed time step.  There's no time-dependent functionality in the quasistatic simulator, so this doesn't matter
    diffs = []
    for frameno in range(n,len(seq)):
        sim.step(control_seq[frameno],dt)
        pred = sim.measurements()['kinova_joint_torques']
        actual = sensor_seq[frameno]['kinova_joint_torques']
        assert len(pred) == len(actual)
        diffs.append(pred-actual)
    diffs = np.array(diffs)
    print("Sequence",seqno,"joint torque RMSEs",np.sqrt(np.mean(diffs**2,axis=0)))

    if seqno >= 4: break

Since comparisons are done frequently, we also have standard functions for calculating these metrics.

In [ ]:
from vsf.sim.metrics import rmse_sensors
calibrators = {'kinova_joint_torques':calibrator}
rmses = rmse_sensors(sim,[dataset[i] for i in range(5)],dataset_config,calibrators)
for i in range(5):
    print("Sequence",i,"joint torque RMSEs",rmses['kinova_joint_torques'][i])
sim.vsf_objects['vsf'].vsf_model = vsf_empty
print("Original RMSE train",rmse_sensors(sim,[dataset[i] for i in range(5)],dataset_config,calibrators,aggregate_seqs=True,aggregate_channels=True)['kinova_joint_torques'])
print("Original RMSE test",rmse_sensors(sim,[dataset[i] for i in range(5,30)],dataset_config,calibrators,aggregate_seqs=True,aggregate_channels=True)['kinova_joint_torques'])
sim.vsf_objects['vsf'].vsf_model = vsf_est
print("Estimated RMSE train",rmse_sensors(sim,[dataset[i] for i in range(5)],dataset_config,calibrators,aggregate_seqs=True,aggregate_channels=True)['kinova_joint_torques'])
print("Estimated RMSE test",rmse_sensors(sim,[dataset[i] for i in range(5,30)],dataset_config,calibrators,aggregate_seqs=True,aggregate_channels=True)['kinova_joint_torques'])

Now we see that we've done a much better job of estimating the RMSEs on the training set but we haven't really improved the testing RMSEs very much.  This is because we've only used 5 sequences for training, and they may have not done a good job of covering the object.  This is a typical issue with using touch data alone.  Read on to the next section to explore better ways of extrapolating using vision features and priors. 

## Extrapolating VSF data by color

This section provides a basic demonstration of learning a color-conditioned stiffness prior.
We use a simple algorithm that naively regresses the estimated VSF stiffness using stochastic gradient descent.
For more advanced and accurate prior learning, please refer to the Bayesian meta-learning function in estimator module.

In [ ]:
import torch
from vsf.prior.conditional_distribution import LinearGaussianPriorConfig, LinearGaussianPrior
from vsf.prior.prior_factory import LearnableVSFPriorFactory

linear_gaussian_condig = LinearGaussianPriorConfig(c_dim=3, diag=True, non_neg=True)

# setup a simple meta-prior using linear Gaussian with RGB as input features
prior_factory = LearnableVSFPriorFactory(['color'], LinearGaussianPrior(linear_gaussian_condig))

# TODO: point vsf does not have consistent dtype, need manually convert to float
vsf_est.features['color'] = vsf_est.features['color'].float()

prior_factory.meta_learn(vsfs=[vsf_est], features=[{'color':vsf_est.features['color'].float()}])

In [ ]:
# Now create an estimator using the learned prior
estimator = PointVSFEstimator(PointVSFEstimatorConfig(), prior_factory)

t0 = time.time()
estimator.batch_estimate(sim, vsf_est, [dataset[i] for i in range(5)], dataset_config, {sensor.name:TareCalibrator()})
t1 = time.time()
print("Estimation took time",t1-t0)

In [ ]:
# show results with color prior, we expect to see a larger high stiffness region 
vsf_show(vsf_est)

## Estimating VSFs for bodies that move

We need to load the brown_boot_moving object for estimating movable VSF.

In [2]:
from vsf import vsf_from_file
moving_vsf_empty = vsf_from_file('../demo_data/saved_vsfs/brown_boot_moving/vsf_empty.npz')

In [3]:
# Show the processed VSF model
from vsf.visualize.klampt_visualization import vsf_to_point_cloud
vis.debug(empty_pc = vsf_to_point_cloud(moving_vsf_empty, masked_view_fraction=1.0))

Stiffness range:  0.0 0.0
################################################################
klampt.vis: Running multi-threaded dialog, waiting to complete...


vis: creating GL window
######### QtGLWindow setProgram ###############
######### QtGLWindow setProgram ###############
#########################################
klampt.vis: Dialog on window 1 starting
#########################################
######### QtGLWindow Initialize GL ###############
QtGLWidget.initialize: no action menu?
QtGLWindow.resizeGL: called when invisible?
QtGLWindow.paintGL: called while invisible?
#########################################
klampt.vis: Dialog done on window 1 result 0
#########################################
klampt.vis: ... dialog done, leaving thread open
################################################################


vis: creating GL window
######### QtGLWindow setProgram ###############
#########################################
klampt.vis: Making window 0
#########################################
######### QtGLWindow Initialize GL ###############
QtGLWindow.resizeGL: called when invisible?
QtGLWindow.paintGL: called while invisible?
#########################################
klampt.vis: Window 0 close
#########################################


In [4]:
from vsf.sim import klamptWorldWrapper, QuasistaticVSFSimulator
from vsf.sim.point_vsf_body import ContactParams
from vsf.sensor.punyo_dense_force_sensor import PunyoDenseForceSensor

# create a new world
world = klamptWorldWrapper()
world.add_robot('kinova','../knowledge/robot_model/kinova_gen3.urdf')
robot = world.world.robot(0)

# the punyo geometry is added to the end effector link of the robot
world.add_geometry_from_file('punyo', '../knowledge/robot_model/punyo_mesh_complete.ply', 
                             geom_type='deformable', parent_name='EndEffector_Link', 
                             parent_relative_transform=np.array([[0.0, -1.0, 0.0, 0.0], [0.34202014, 0.0, -0.93969262, 0.02111], 
                                                                 [0.93969262, 0.0, 0.34202014, 0.096142], [0.0, 0.0, 0.0, 1.0]]))

# this preprocessing needs to be done before running the simulator with a point VSF body
world.setup_local_pcd_lst('open3d')
world.setup_local_sdf_lst()

# initialize all sensors to add in the simulator
dense_force_sensor = PunyoDenseForceSensor('punyo_force','punyo')

# create a simulator with the world, a joint torque sensor, and the vsf body
moving_sim = QuasistaticVSFSimulator(world, [dense_force_sensor])
vsf_body = moving_sim.add_deformable('vsf', moving_vsf_empty, contact_params=None)  #if you want to customize how the VSF is simulated, you can pass in a ContactParams object here

WorldModel::LoadRobot: ../knowledge/robot_model/kinova_gen3.urdf
URDFParser: Link size: 9
URDFParser: Joint size: 9
URDFParser: Done loading robot file ../knowledge/robot_model/kinova_gen3.urdf
FMM_Fill identifies 1305 surface, 731 interior, 2003 exterior cells
FMM starting with 835 surface cells, grid of size 13 13 21
FMM found 1151 interior and 2398 exterior cells
Geometry: AnyGeometry::Convert: FMM grid bounding box -0.0648063 -0.0649216 -0.019575   0.0651937 0.0650784 0.190425
FMM_Fill identifies 1076 surface, 539 interior, 1737 exterior cells
FMM starting with 684 surface cells, grid of size 13 13 21
FMM found 867 interior and 2682 exterior cells
Geometry: AnyGeometry::Convert: FMM grid bounding box -0.065 -0.0662972 -0.193375   0.065 0.0637028 0.016625
FMM_Fill identifies 1503 surface, 788 interior, 2376 exterior cells
FMM starting with 829 surface cells, grid of size 13 31 13
FMM found 1113 interior and 4126 exterior cells
Geometry: AnyGeometry::Convert: FMM grid bounding box -0

Setup the dataset with moving object

In [5]:
from vsf.dataset.constructors import DatasetConfig, dataset_from_config

keys = {'kinova_state': 7, 
        'punyo_state': [[390, 3], np.ndarray], 
        'punyo_force': [[390, 3], np.ndarray], 
        'vsf_state': [[4, 4], np.ndarray]}  # vsf_state is rigid transformation of VSF object
dataset_config = DatasetConfig(type='moving_dataset', 
                               path='../demo_data/datasets/brown_boot_moving', keys=keys)
dataset = dataset_from_config(dataset_config)
print("Dataset has {} sequences".format(len(dataset)))

Dataset has 90 sequences


Run point VSF estimation on moving object. Since we use the Punyo dense force sensor, the estimation time is expected longer.

In [6]:
from vsf.sensor import TareCalibrator
from vsf.estimator.point_vsf_estimator import PointVSFEstimator, PointVSFEstimatorConfig
from vsf.prior.prior_factory import GaussianVSFPriorFactory
import time
import copy
moving_vsf_est = moving_vsf_empty
moving_vsf_empty = copy.deepcopy(moving_vsf_est)  #save a copy of the uninitialized VSF for later

#the second argument sets the prior estimate to a mean of 0.5 and a standard deviation of 1.0
estimator = PointVSFEstimator(PointVSFEstimatorConfig(), GaussianVSFPriorFactory(0.0,1.0))

t0 = time.time()
estimator.batch_estimate(moving_sim, moving_vsf_est, [dataset[i] for i in range(5)], dataset_config)
t1 = time.time()
print("Estimation took time",t1-t0)

Loading sequence ../demo_data/datasets/brown_boot_moving/seq_000
Loading sequence ../demo_data/datasets/brown_boot_moving/seq_001
Loading sequence ../demo_data/datasets/brown_boot_moving/seq_002
Loading sequence ../demo_data/datasets/brown_boot_moving/seq_003
Loading sequence ../demo_data/datasets/brown_boot_moving/seq_004
Generating simulation cache
Number of observations: 5
Observed indices: 40079
QuadProgOptimizer: start solving cvxpy problem...
QuadProgOptimizer: cvxpy solve time = 144.79318618774414
Estimation took time 193.62956285476685


In [7]:
# visualize estimated vsf
vsf_show(moving_vsf_est)

Setting voxel grid resolution 0.0019416289327088357
vsf_show: Using the following stiffness values: [2.6494479353489364e-12, 0.0014182167360559106, 0.002836433472111821, 0.004254650324583054, 0.005672866944223642]
klampt.vis: auto-fitting camera to scene.


Unrecognized OpenGL version
Unrecognized OpenGL version


## Multi-sensor estimation

The estimation techniques above can work for any number of sensors.  Simply create more sensors to add to the simulator, and set up the observations dictionary to the estimator appropriately.    

In [2]:
from vsf import vsf_from_file
shoe_vsf_empty = vsf_from_file('../demo_data/saved_vsfs/white_nike_fixed/vsf_empty.npz')

In [ ]:
# Show the processed VSF model
from vsf.visualize.klampt_visualization import vsf_to_point_cloud
vis.debug(empty_pc = vsf_to_point_cloud(shoe_vsf_empty, masked_view_fraction=1.0))

Stiffness range:  0.0 0.0
################################################################
klampt.vis: Running multi-threaded dialog, waiting to complete...


vis: creating GL window
######### QtGLWindow setProgram ###############
######### QtGLWindow setProgram ###############
#########################################
klampt.vis: Dialog on window 1 starting
#########################################
######### QtGLWindow Initialize GL ###############
QtGLWidget.initialize: no action menu?
QtGLWindow.resizeGL: called when invisible?
QtGLWindow.paintGL: called while invisible?
#########################################
klampt.vis: Dialog done on window 1 result 0
#########################################
klampt.vis: ... dialog done, leaving thread open
################################################################


vis: creating GL window
######### QtGLWindow setProgram ###############
#########################################
klampt.vis: Making window 0
#########################################
######### QtGLWindow Initialize GL ###############
QtGLWindow.resizeGL: called when invisible?
QtGLWindow.paintGL: called while invisible?
#########################################
klampt.vis: Window 0 close
#########################################
vis: creating GL window
######### QtGLWindow setProgram ###############
######### QtGLWindow setProgram ###############
#########################################
klampt.vis: Dialog on window 2 starting
#########################################
######### QtGLWindow Initialize GL ###############
QtGLWidget.initialize: no action menu?
QtGLWindow.resizeGL: called when invisible?
QtGLWindow.paintGL: called while invisible?
#########################################
klampt.vis: Dialog done on window 2 result 0
#########################################


In [4]:
from vsf.sim import klamptWorldWrapper, QuasistaticVSFSimulator
from vsf.sim.point_vsf_body import ContactParams
from vsf.sensor.joint_torque_sensor import JointTorqueSensor
from vsf.sensor.punyo_pressure_sensor import PunyoPressureSensor

# create a new world
world = klamptWorldWrapper()
world.add_robot('kinova','../knowledge/robot_model/kinova_gen3.urdf')
robot = world.world.robot(0)

# the punyo geometry is added to the end effector link of the robot
world.add_geometry_from_file('punyo', '../knowledge/robot_model/punyo_mesh_complete.ply', 
                             geom_type='deformable', parent_name='EndEffector_Link', 
                             parent_relative_transform=np.array([[0.0, -1.0, 0.0, 0.0], 
                                                                 [-0.05582150, 0.0, -0.9984407, 0.02111], 
                                                                 [0.99844076, 0.0, -0.0558215, 0.096142], 
                                                                 [0.0, 0.0, 0.0, 1.0]]))

# this preprocessing needs to be done before running the simulator with a point VSF body
world.setup_local_pcd_lst('open3d')
world.setup_local_sdf_lst()

# initialize all sensors to add in the simulator
joint_torque_sensor = JointTorqueSensor('kinova_joint_torques','kinova',[robot.link(i).name for i in range(1,8)])
pressure_sensor = PunyoPressureSensor('punyo_pressure','punyo')

# create a simulator with the world, a joint torque sensor, and the vsf body
shoe_sim = QuasistaticVSFSimulator(world, [joint_torque_sensor, pressure_sensor])
vsf_body = shoe_sim.add_deformable('vsf', shoe_vsf_empty, contact_params=None)  #if you want to customize how the VSF is simulated, you can pass in a ContactParams object here

WorldModel::LoadRobot: ../knowledge/robot_model/kinova_gen3.urdf
URDFParser: Link size: 9
URDFParser: Joint size: 9
URDFParser: Done loading robot file ../knowledge/robot_model/kinova_gen3.urdf
FMM_Fill identifies 1305 surface, 731 interior, 2003 exterior cells
FMM starting with 835 surface cells, grid of size 13 13 21
FMM found 1151 interior and 2398 exterior cells
Geometry: AnyGeometry::Convert: FMM grid bounding box -0.0648063 -0.0649216 -0.019575   0.0651937 0.0650784 0.190425
FMM_Fill identifies 1076 surface, 539 interior, 1737 exterior cells
FMM starting with 684 surface cells, grid of size 13 13 21
FMM found 867 interior and 2682 exterior cells
Geometry: AnyGeometry::Convert: FMM grid bounding box -0.065 -0.0662972 -0.193375   0.065 0.0637028 0.016625
FMM_Fill identifies 1503 surface, 788 interior, 2376 exterior cells
FMM starting with 829 surface cells, grid of size 13 31 13
FMM found 1113 interior and 4126 exterior cells
Geometry: AnyGeometry::Convert: FMM grid bounding box -0

Setup the dataset for multiple sensors.

In [5]:
from vsf.dataset.constructors import DatasetConfig, dataset_from_config

keys = {'kinova_joint_torques': 7, 'kinova_state': 7, 'punyo_pressure': 1}  #describes the keys present in the dataset
dataset_config = DatasetConfig(type='multi_sensor_dataset', 
                               path='../demo_data/datasets/white_nike_fixed', keys=keys)
dataset = dataset_from_config(dataset_config)
print("Dataset has {} sequences".format(len(dataset)))

Dataset has 181 sequences


In [6]:
from vsf.sensor import TareCalibrator
from vsf.estimator.point_vsf_estimator import PointVSFEstimator, PointVSFEstimatorConfig
from vsf.prior.prior_factory import GaussianVSFPriorFactory
import time
import copy
shoe_vsf_est = shoe_vsf_empty
shoe_vsf_empty = copy.deepcopy(shoe_vsf_est)  #save a copy of the uninitialized VSF for later

#the second argument sets the prior estimate to a mean of 0.5 and a standard deviation of 1.0
estimator = PointVSFEstimator(PointVSFEstimatorConfig(), GaussianVSFPriorFactory(0.0,1.0))

calibrators = {'kinova_joint_torques':TareCalibrator(), 'punyo_pressure':TareCalibrator()}

t0 = time.time()
estimator.batch_estimate(shoe_sim, shoe_vsf_est, [dataset[i] for i in range(5)], dataset_config, calibrators)
t1 = time.time()
print("Estimation took time",t1-t0)

Loading sequence ../demo_data/datasets/white_nike_fixed/seq_000
Loading sequence ../demo_data/datasets/white_nike_fixed/seq_001
Loading sequence ../demo_data/datasets/white_nike_fixed/seq_002
Loading sequence ../demo_data/datasets/white_nike_fixed/seq_003
Loading sequence ../demo_data/datasets/white_nike_fixed/seq_004
Generating simulation cache


/home/motion/openvsf/demos/../vsf/sensor/base_calibrator.py:78: UserWarning: Contact happens at the first sample, please check the command sequence
  warnings.warn('Contact happens at the first sample, please check the command sequence')


Number of observations: 5
Observed indices: 1939
QuadProgOptimizer: start solving cvxpy problem...
QuadProgOptimizer: cvxpy solve time = 0.3820312023162842
Estimation took time 0.8587646484375


In [7]:
# visualize estimated vsf
vsf_show(shoe_vsf_est)

Setting voxel grid resolution 0.008660254037844345
vsf_show: Using the following stiffness values: [2.05728468927191e-07, 0.05471273139119148, 0.10942526161670685, 0.16413778066635132, 0.21885031461715698]
klampt.vis: auto-fitting camera to scene.


Unrecognized OpenGL version
Unrecognized OpenGL version


In [9]:
from vsf.utils.data_utils import remap_dict_in_seq

for seqno in range(len(dataset)):
    #extract the sequence of controls and observations.  This is boilerplate
    seq = dataset[seqno]
    
    control_seq, sensor_seq = remap_dict_in_seq(seq, shoe_sim.get_control_keys(), shoe_sim.get_sensor_keys())
        
    #run the calibration
    shoe_sim.reset()
    n = max(calibrators['kinova_joint_torques'].calibrate(joint_torque_sensor,shoe_sim,control_seq,sensor_seq), 
            calibrators['punyo_pressure'].calibrate(pressure_sensor,shoe_sim,control_seq,sensor_seq))

    #now, run the simulator and compare the predicted torques to the actual torques
    dt = 0.1  # a guessed time step.  There's no time-dependent functionality in the quasistatic simulator, so this doesn't matter
    joint_torque_diffs = []
    punyo_pressure_diffs = []
    for frameno in range(n,len(seq)):
        shoe_sim.step(control_seq[frameno],dt)
        joint_torque_pred = shoe_sim.measurements()['kinova_joint_torques']
        joint_torque_actual = sensor_seq[frameno]['kinova_joint_torques']
        assert len(joint_torque_pred) == len(joint_torque_actual)
        joint_torque_diffs.append(joint_torque_pred-joint_torque_actual)
        
        punyo_pressure_pred = shoe_sim.measurements()['punyo_pressure']
        punyo_pressure_actual = sensor_seq[frameno]['punyo_pressure']
        assert len(punyo_pressure_pred) == len(punyo_pressure_actual)
        punyo_pressure_diffs.append(punyo_pressure_pred-punyo_pressure_actual)

    joint_torque_diffs = np.array(joint_torque_diffs)
    print("Sequence",seqno,"joint torque RMSEs",np.sqrt(np.mean(joint_torque_diffs**2,axis=0)))
    
    punyo_pressure_diffs = np.array(punyo_pressure_diffs)
    print("Sequence",seqno,"punyo pressure RMSEs",np.sqrt(np.mean(punyo_pressure_diffs**2,axis=0)))

    if seqno >= 4: break

/home/motion/openvsf/demos/../vsf/sensor/base_calibrator.py:78: UserWarning: Contact happens at the first sample, please check the command sequence
  warnings.warn('Contact happens at the first sample, please check the command sequence')
/tmp/ipykernel_535345/631405553.py:23: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  joint_torque_diffs.append(joint_torque_pred-joint_torque_actual)
/tmp/ipykernel_535345/631405553.py:28: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  punyo_pressure_diffs.append(punyo_pressure_pred-punyo_pressure_actual)


Sequence 0 joint torque RMSEs [3.38299917 4.10381759 2.24333016 4.73418243 0.42585634 2.76030298
 0.24903376]
Sequence 0 punyo pressure RMSEs [0.48944667]
Sequence 1 joint torque RMSEs [2.5188111  4.03694494 2.69195095 2.39329193 0.17609377 2.22228504
 0.18011346]
Sequence 1 punyo pressure RMSEs [0.38843873]
Sequence 2 joint torque RMSEs [2.45111927 4.85118778 1.20088361 5.96571212 0.20308004 2.94272546
 0.40444197]
Sequence 2 punyo pressure RMSEs [0.34885589]
Sequence 3 joint torque RMSEs [1.44332113 4.55244675 0.10052913 3.69278201 0.15918021 2.63158238
 0.17473143]
Sequence 3 punyo pressure RMSEs [0.3691069]
Sequence 4 joint torque RMSEs [ 3.53635645  5.50165009  1.36797794 10.725968    0.47142986  4.44815498
  0.36708171]
Sequence 4 punyo pressure RMSEs [0.84524727]
